# Example ticket generation workflow

This notebook sets up an environment, queries mediacloud for sources, creates Source objects and runs 'issues' on them, then generates tickets in zammad out of the detected issues

In [1]:
%pip install -q -e  ../. 
import os

mc_token = "BYO"
os.environ["MC_API_TOKEN"] = mc_token
os.environ["ZAMMAD_TOKEN"] = "BYO"

from directory_issues import Collection, Source
from directory_issues.clients.zammad_client import ZammadClient
from IPython.display import HTML


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Use the mediacloud directory to get a specific subset of sources
In this case, those that have a defined 'url search string'


In [2]:
import mediacloud
mc_dir = mediacloud.api.DirectoryApi(mc_token)
sources_with_url_search_strings = []

more_sources = True
offset = 0
limit=100000
while more_sources and len(sources_with_url_search_strings) < 200:
    
    resp = mc_dir.source_list(limit = limit, offset=offset)
    matches = [r for r in resp["results"] if r["url_search_string"] is not None]
    S = [Source(r) for r in matches]
    for s in S:
        print(s)
        s.find_issues()
    sources_with_url_search_strings.extend(S)
    print(len(sources_with_url_search_strings))
    if resp["next"] is not None:
        offset += limit
    else:
        more_sources = False
        

Source(indiatimes.com, http://timesofindia.indiatimes.com/)
Source(ctvnews.ca, http://bc.ctvnews.ca/)
Source(theguardian.com, https://www.theguardian.com/us)
Source(intoday.in, http://indiatoday.intoday.in/#spider)
Source(cbslocal.com, https://www.cbsnews.com/boston/)
Source(forward.com, http://forward.com)
Source(bizjournals.com, http://www.bizjournals.com/boston)
Source(uol.com.br, http://www.folha.uol.com.br/)
Source(iol.co.za, http://www.iol.co.za/the-star)
Source(cbslocal.com, http://sanfrancisco.cbslocal.com)
Source(bizjournals.com, http://www.bizjournals.com/sanfrancisco/)
Source(bdlive.co.za, http://www.bdlive.co.za/)
Source(ctvnews.ca, http://regina.ctvnews.ca/)
Source(ctvnews.ca, http://windsor.ctvnews.ca/)
Source(news12.com, http://connecticut.news12.com/)
Source(nation.co.ke, http://nairobinews.nation.co.ke)
Source(cbslocal.com, http://newyork.cbslocal.com)
Source(india.com, http://zeenews.india.com)
Source(cbslocal.com, http://losangeles.cbslocal.com)
Source(cbslocal.com, 

## A demo source ticket with these issues


In [3]:
sources_with_issues = [s for s in sources_with_url_search_strings if len(s.source_issues) != 0]
source = sources_with_issues[10]
print(source.source_data.id)
print(source.source_data.label)
message = source.render_template()
HTML(message)


42021
echomsk


## Using the zamad client to create tickets 
(this is likely to be eventually wrapped up within a base function)

In [4]:
zc = ZammadClient()


zammad_token='z6HEXJ14ZkuTeyBMhJ8UJqKqlRQ7qT_tTK0wOMK2-aW_dy6TXRxhKw91mpewt4GL' zammad_url='https://support.mediacloud.org/api/v1/' default_zammad_group='Users' default_zammad_user='Directory Detective' default_zammad_email='support@mediacloud.org'


In [8]:
for source in sources_with_issues:
    message = source.render_template()
    zc.source_ticket(
        message,
        source.source_data.label,
        source.source_data.id,
        [c["id"] for c in source.collections]
    )